In [2]:
import os
os.getcwd()
os.chdir("H:\pyspark-coding-interview")
os.getcwd()

'H:\\pyspark-coding-interview'

In [1]:
from pyspark.sql import SparkSession

# Create a Spark session with optimized settings
spark = (
    SparkSession.builder 
    .appName("OptimizedLocalSpark") 
    .config("spark.driver.memory", "8g")        
    .config("spark.executor.memory", "8g")    
    .config("spark.executor.cores", "4")       
    .config("spark.cores.max", "12")           
    .config("spark.sql.shuffle.partitions", "28")  
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") 
    .getOrCreate()
)
sc = spark.sparkContext
# Check Configuration
print(sc.getConf().getAll())

[('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'), ('spark.app.submitTime', '1729292465803'), ('spark.executor.id', 'driver'), ('spark.driver.host', 'localh

+----------+--------+-----+
|      date|category|sales|
+----------+--------+-----+
|2023-10-01|       A|  100|
|2023-10-02|       A|  200|
|2023-10-03|       A|  300|
|2023-10-01|       B|  150|
|2023-10-02|       B|   50|
|2023-10-03|       B|  100|
|2023-10-01|       C|  200|
|2023-10-02|       C|  300|
|2023-10-03|       C|  400|
+----------+--------+-----+



In [6]:


# Sample data for 20 records based on the visible columns from the image
data = [
    ("Guy", "Gilbert", "guy1@adventure-works.com", "320-555-0195", "M", "Production", 400000),
    ("Kevin", "Brown", "kevin@adventure-works.com", "150-555-0189", "M", "Marketing", 700000),
    ("Roberto", "Tamburello", "roberto@adventure-works.com", "212-555-0187", "M", "Engineering", 800000),
    ("Renee", "Lewis", "renee.lewis@adventure-works.com", "456-839-7190", "F", "Marketing", 700000),
    ("Renee", "Thomas", "renee.thomas@adventure-works.com", "467-999-1000", "F", "Production", 600000),
    ("Thierry", "D'Hers", "thierry@adventure-works.com", "168-555-0183", "M", "Tool Design", 800000),
    ("Guy", "Gilbert", "guy@adventure-works.com", None, None, None, None),
    ("JoLynn", "Dobney", "jolynn@adventure-works.com", "903-555-0145", "F", "Production", 100000),
    ("Ruth", "Elberstock", "ruth@adventure-works.com", "145-555-0130", "F", "Production", 300000),
    ("Gail", "Erickson", "gail@adventure-works.com", "849-555-0139", "F", "Engineering", 300000),
    ("Barry", "Johnson", "barry@adventure-works.com", "206-555-0180", "M", "Production", 900000),
    ("Alice", "Smith", "alice@adventure-works.com", "321-555-0145", "F", "HR", 500000),
    ("John", "Doe", "john@adventure-works.com", "312-555-0199", "M", "IT", 450000),
    ("Emily", "Davis", "emily@adventure-works.com", "908-555-0101", "F", "Sales", 550000),
    ("James", "Bond", "james@adventure-works.com", "007-555-0707", "M", "Spy", 1000000),
    ("Olivia", "Jones", "olivia@adventure-works.com", "444-555-0198", "F", "Finance", 700000),
    ("Sophia", "Miller", "sophia@adventure-works.com", "555-555-0123", "F", "Marketing", 650000),
    ("Liam", "Wilson", "liam@adventure-works.com", "666-555-0188", "M", "Engineering", 800000),
    ("Noah", "Moore", "noah@adventure-works.com", "777-555-0155", "M", "Production", 750000),
    ("Mia", "Taylor", "mia@adventure-works.com", "888-555-0190", "F", "Production", 620000)
]

# Define column names
columns = ["FirstName", "LastName", "EmailAddress", "Phone", "Gender", "DepartmentName", "Salary"]

# Create DataFrame
df = spark.createDataFrame(data, columns)
df.cache()
# Show the DataFrame
df.createOrReplaceTempView("Employee")
df.show()


+---------+----------+--------------------+------------+------+--------------+-------+
|FirstName|  LastName|        EmailAddress|       Phone|Gender|DepartmentName| Salary|
+---------+----------+--------------------+------------+------+--------------+-------+
|      Guy|   Gilbert|guy1@adventure-wo...|320-555-0195|     M|    Production| 400000|
|    Kevin|     Brown|kevin@adventure-w...|150-555-0189|     M|     Marketing| 700000|
|  Roberto|Tamburello|roberto@adventure...|212-555-0187|     M|   Engineering| 800000|
|    Renee|     Lewis|renee.lewis@adven...|456-839-7190|     F|     Marketing| 700000|
|    Renee|    Thomas|renee.thomas@adve...|467-999-1000|     F|    Production| 600000|
|  Thierry|    D'Hers|thierry@adventure...|168-555-0183|     M|   Tool Design| 800000|
|      Guy|   Gilbert|guy@adventure-wor...|        null|  null|          null|   null|
|   JoLynn|    Dobney|jolynn@adventure-...|903-555-0145|     F|    Production| 100000|
|     Ruth|Elberstock|ruth@adventure-wo...|

# Using Spark SQL


In [19]:
res = spark.sql("""
select 
DepartmentName,
FirstName,
LastName,
Salary, 
SUM(Salary) OVER (Partition by DepartmentName ORDER BY salary desc rows between unbounded preceding and current row  ) as running_total,
SUM(Salary) OVER (ORDER BY Salary DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS total_running_sum_over_entire_dataset
from Employee
order by running_total
""")
res.show()

+--------------+---------+----------+-------+-------------+-------------------------------------+
|DepartmentName|FirstName|  LastName| Salary|running_total|total_running_sum_over_entire_dataset|
+--------------+---------+----------+-------+-------------+-------------------------------------+
|          null|      Guy|   Gilbert|   null|         null|                             11620000|
|            IT|     John|       Doe| 450000|       450000|                             10520000|
|            HR|    Alice|     Smith| 500000|       500000|                             10070000|
|         Sales|    Emily|     Davis| 550000|       550000|                              9570000|
|       Finance|   Olivia|     Jones| 700000|       700000|                              7150000|
|     Marketing|    Kevin|     Brown| 700000|       700000|                              5750000|
|   Engineering|  Roberto|Tamburello| 800000|       800000|                              2700000|
|   Tool Design|  Th

In [11]:
result = spark.sql("""
    SELECT 
        e1.FirstName, 
        e1.LastName, 
        e1.DepartmentName, 
        e1.Salary,
        SUM(e2.Salary) AS cumulative_sum
    FROM employee e1
    JOIN employee e2 ON e1.Salary >= e2.Salary
    GROUP BY e1.FirstName, e1.LastName, e1.DepartmentName, e1.Salary
    ORDER BY e1.Salary DESC
""")

result.show()

+---------+----------+--------------+-------+--------------+
|FirstName|  LastName|DepartmentName| Salary|cumulative_sum|
+---------+----------+--------------+-------+--------------+
|    James|      Bond|           Spy|1000000|      11620000|
|    Barry|   Johnson|    Production| 900000|      10620000|
|  Roberto|Tamburello|   Engineering| 800000|       9720000|
|  Thierry|    D'Hers|   Tool Design| 800000|       9720000|
|     Liam|    Wilson|   Engineering| 800000|       9720000|
|     Noah|     Moore|    Production| 750000|       7320000|
|    Kevin|     Brown|     Marketing| 700000|       6570000|
|    Renee|     Lewis|     Marketing| 700000|       6570000|
|   Olivia|     Jones|       Finance| 700000|       6570000|
|   Sophia|    Miller|     Marketing| 650000|       4470000|
|      Mia|    Taylor|    Production| 620000|       3820000|
|    Renee|    Thomas|    Production| 600000|       3200000|
|    Emily|     Davis|         Sales| 550000|       2600000|
|    Alice|     Smith|  

# Using Pyspark

In [20]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum, col

df.show()

windowSpec = Window.partitionBy(col("DepartmentName")).orderBy(col("Salary").asc()).rowsBetween(Window.unboundedPreceding, Window.currentRow)
windowSpecAgg = Window.orderBy(col("Salary").asc()).rowsBetween(Window.unboundedPreceding, Window.currentRow)
df5 = df.withColumn("running_sum_over_entire_dataset", sum("Salary").over(windowSpecAgg)).withColumn("running_sum_over_department", sum("Salary").over(windowSpec))
df5.show()




+---------+----------+--------------------+------------+------+--------------+-------+
|FirstName|  LastName|        EmailAddress|       Phone|Gender|DepartmentName| Salary|
+---------+----------+--------------------+------------+------+--------------+-------+
|      Guy|   Gilbert|guy1@adventure-wo...|320-555-0195|     M|    Production| 400000|
|    Kevin|     Brown|kevin@adventure-w...|150-555-0189|     M|     Marketing| 700000|
|  Roberto|Tamburello|roberto@adventure...|212-555-0187|     M|   Engineering| 800000|
|    Renee|     Lewis|renee.lewis@adven...|456-839-7190|     F|     Marketing| 700000|
|    Renee|    Thomas|renee.thomas@adve...|467-999-1000|     F|    Production| 600000|
|  Thierry|    D'Hers|thierry@adventure...|168-555-0183|     M|   Tool Design| 800000|
|      Guy|   Gilbert|guy@adventure-wor...|        null|  null|          null|   null|
|   JoLynn|    Dobney|jolynn@adventure-...|903-555-0145|     F|    Production| 100000|
|     Ruth|Elberstock|ruth@adventure-wo...|